# Getting the data

In [1]:
import numpy as np
import sys
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

# import pandas
import json

# from sklearn.externals import joblib
import pickle
from importlib.resources import path
from pathlib import Path

import numpy as np
import pandas as pd
import sklearn.utils
from pandas import Int64Index, MultiIndex
from sklearn import metrics

#### Trainset

In [2]:
import os
import importlib
import time
import pandas as pd

dirname = os.path.dirname('D:/fincrime-federated/')
os.chdir(dirname)
print(os.getcwd())

import submission_src.fincrime.solution_centralized as funcs
importlib.reload(funcs) 

start_time = time.time()

model_dir = 'D:/fincrime-federated/model/fincrime'
preds_format_path = 'D:/fincrime-federated/prediction/fincrime/prediction_format'
preds_dest_path = 'D:/fincrime-federated/prediction/fincrime/prediction'


## train on data
datapathjsonString = 'data/fincrime/centralized/train/data.json'
swift_data_path = funcs.json_to_dict(datapathjsonString)['swift_data_path']
bank_data_path = funcs.json_to_dict(datapathjsonString)['bank_data_path']

# trainset = funcs.fit(swift_data_path = swift_data_path,
#                     bank_data_path = bank_data_path,
#                     model_dir = model_dir,
#                     preds_format_path = preds_format_path,
#                     preds_dest_path = preds_dest_path,
#                     m = 'xgboost')
def json_to_dict(datapathjsonString):
    datapathJson = open(datapathjsonString)
    datapathDict = json.load(datapathJson)
    return datapathDict


def load_data(swift_data_path, bank_data_path):
    swift_data = pd.read_csv(swift_data_path, index_col="MessageId")
    swift_data["Timestamp"] = swift_data["Timestamp"].astype("datetime64[ns]")
    bank_data = pd.read_csv(bank_data_path)
    return swift_data, bank_data

train_data, bank_data = load_data(
    swift_data_path=swift_data_path, bank_data_path=bank_data_path
)

# Merging with bank details
train_data = pd.merge(
    train_data,
    bank_data,
    left_on="OrderingAccount",
    right_on="Account",
    how="left",
)
train_data = pd.merge(
    train_data,
    bank_data,
    left_on="BeneficiaryAccount",
    right_on="Account",
    how="left",
    suffixes=["_order", "_ben"],
)
trainset = train_data
print(trainset.head())

D:\fincrime-federated
   Timestamp                                  UETR    Sender  Receiver  \
0 2022-01-01  f474fdb3-4675-4fff-ab7e-3469f82bd6a7  DPSUFRPP  ABVVUS6S   
1 2022-01-01  c9158def-dab1-4bfb-a31f-7f51c6679d60  BRRGPTPL  CBLHESMM   
2 2022-01-01  d371ba0a-823f-4243-98ba-94ff18523420  BRRGPTPL  CBLHESMM   
3 2022-01-01  5a53a257-4dc9-4800-abb2-4cd1d55c8345  DPSUFRPP  ABVVUS6S   
4 2022-01-01  f27867ac-35e2-46af-8248-0a2d0d9bf00d  DPSUFRPP  ABVVUS6S   

  TransactionReference         OrderingAccount                  OrderingName  \
0    PETX22-FXIDA-7054  FR90714755422956984353         PHACELIA HETEROPHYLLA   
1    PETX22-NO-FX-1736   PT8895792452733129969     GONOLOBUS STEPHANOTRICHUS   
2    PETX22-NO-FX-1687  PT92895792452733126420  LECHEA INTERMEDIA-INTERMEDIA   
3     DPSU22-FXIYA-517      358727697099645998                   SCLERANTHUS   
4   PETX22-FXIDA-11878  FR71714755422956985471          SELAGINELLA ASPRELLA   

           OrderingStreet       OrderingCountryCityZ

#### Testset

In [3]:
# predict on test data
import submission_src.fincrime.solution_centralized as funcs
importlib.reload(funcs) 

datapathjsonString = 'data/fincrime/centralized/test/data.json'
swift_data_path = funcs.json_to_dict(datapathjsonString)['swift_data_path']
bank_data_path = funcs.json_to_dict(datapathjsonString)['bank_data_path']


test_data, bank_data = load_data(
    swift_data_path=swift_data_path, bank_data_path=bank_data_path
)

# Merging with bank details
test_data = pd.merge(
    test_data,
    bank_data,
    left_on="OrderingAccount",
    right_on="Account",
    how="left",
)
test_data = pd.merge(
    test_data,
    bank_data,
    left_on="BeneficiaryAccount",
    right_on="Account",
    how="left",
    suffixes=["_order", "_ben"],
)

testset = test_data
print(testset.head())
print("Total time taken ", (time.time() - start_time)," seconds")

            Timestamp                                  UETR    Sender  \
0 2022-01-28 22:15:00  43a12cd8-e14b-4737-a5a4-39b124468509  ZOUOGB22   
1 2022-01-29 05:34:00  ce04b39a-8162-43d6-ab82-f36705d1326a  WVOLDEMM   
2 2022-01-29 07:31:00  8f7fe9e4-b8cf-46fd-9412-e6c6bc6891db  ABVVUS6S   
3 2022-01-30 23:07:00  037045bd-ea1b-4b1e-bc86-9c3c287353be  CBLHESMM   
4 2022-01-29 09:24:00  6d17c9e0-42f5-4081-a3f7-49b2a85cc7c8  ZOUOGB22   

   Receiver TransactionReference         OrderingAccount  \
0  FJKNESMM   PETX22-FXIDA-11158  GB43679059808029769468   
1  MBYOTRIS     PETX22-NO-FX-996  DE50020224692198579962   
2  WVOLDEMM     PETX22-FXIGA-828             6.11024E+17   
3  FJKNESMM    PETX22-NO-FX-1595  ES11897100852916939240   
4  WMVZGB2L    PETX22-FXIDA-9106  GB69679059808029738445   

                     OrderingName                         OrderingStreet  \
0                 SABAL CAUSIARUM            14 CLARE ISLAND JOSHUAMOUTH   
1                  IRIS FERNALDII               

### Dropping Null Values

In [4]:
# trainset.dropna(axis=0,how='any',inplace=True)
# print("trainset shape is:"trainset.shape)
# testset.dropna(axis=0,how='any',inplace=True)
# print("testset shape is:"testset.shape)

### Normalization / Scaling

In [5]:
# def stats(train_dataset):
#     train_stats = train_dataset.describe()
#     train_stats = train_stats.transpose()
#     return train_stats

# def norm(x):
#     train_stats=stats(x)
#     return (x - train_stats['mean']) / train_stats['std']

In [6]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# from sklearn.preprocessing import RobustScaler
# scaler = RobustScaler(quantile_range=(25, 75))

def norm(x):
    scaler.fit(x)
    return scaler.transform(x)

# Data Prep

In [7]:
def freq_by_features(
    df,
    level_1,
    level_2,
    model_dir,
    save_as_object=False,
    map_from_train_set=False,
) -> None:

    if map_from_train_set == True:

        level_1_level_2_frequency = pickle.load(
            open(
                model_dir
                + "/"
                + str(level_1 + "_" + level_2 + "_frequency")
                + ".sav",
                "rb",
            )
        )
        _level_1 = df.loc[:, level_1].unique()
        _level_2 = df.loc[:, level_2].unique()
        df[str(level_1 + "_" + level_2)] = df.loc[:, level_1] + df.loc[
            :, level_2
        ].astype(str)
        df.loc[:, str(level_1 + "_" + level_2 + "_frequency")] = df.loc[
            :, str(level_1 + "_" + level_2)
        ].map(level_1_level_2_frequency)
    else:
        _level_1 = df.loc[:, level_1].unique()
        _level_2 = df.loc[:, level_2].unique()
        df[str(level_1 + "_" + level_2)] = df.loc[:, level_1] + df.loc[
            :, level_2
        ].astype(str)
        level_1_level_2_frequency = {}
        for s in _level_1:
            level_1_rows = df[df[level_1] == s]
            for h in _level_2:
                level_1_level_2_frequency[s + str(h)] = len(
                    level_1_rows[level_1_rows.loc[:, level_2] == h]
                )

        df.loc[:, str(level_1 + "_" + level_2 + "_frequency")] = df.loc[
            :, str(level_1 + "_" + level_2)
        ].map(level_1_level_2_frequency)

        if save_as_object == True:
            pickle.dump(
                level_1_level_2_frequency,
                open(
                    model_dir
                    + "/"
                    + str(level_1 + "_" + level_2 + "_frequency")
                    + ".sav",
                    "wb",
                ),
            )

    ...

In [8]:
def data_prep1(train_data):
    # Feature engineering
#     train_data = train_data.dropna(axis=0)
    
    train_data = create_features(
        df=train_data, model_dir=model_dir, map_from_train_set=False
    )

    # Keep below columns for training and testing
    cols_to_keep = [
        "SettlementAmount",
        "InstructedAmount",
        "Label",
        "hour",
        "Flags_ben",
        "MissingBenAccount",
        "MissingOrdAccount",
        "Sender_hour_frequency",
        # 'sender_currency_amount_average',
        "Sender_Receiver_frequency",
        "Sender_InstructedCurrency_frequency",
        "seq",
        # 'receiver_transactions',
        "Receiver_SettlementCurrency_frequency",
        "Receiver_hour_frequency",
        "DifferentOrderNum",
        "DifferentBenNum",
        "DifferentOrderName",
        "DifferentBenName",
        "DifferentOrderStreet",
        "DifferentBenStreet",
        "DifferentOrderZip",
        "DifferentBenZip",
    ]

    train_data_2 = train_data[cols_to_keep]
    return train_data_2

In [9]:
def data_prep2(train_data):
    # Feature engineering
#     train_data = train_data.dropna(axis=0)
    train_data = create_features(
        df=train_data, model_dir=model_dir, map_from_train_set=True
    )

    # Keep below columns for training and testing
    cols_to_keep = [
        "SettlementAmount",
        "InstructedAmount",
        "Label",
        "hour",
        "Flags_ben",
        "MissingBenAccount",
        "MissingOrdAccount",
        "Sender_hour_frequency",
        # 'sender_currency_amount_average',
        "Sender_Receiver_frequency",
        "Sender_InstructedCurrency_frequency",
        "seq",
        # 'receiver_transactions',
        "Receiver_SettlementCurrency_frequency",
        "Receiver_hour_frequency",
        "DifferentOrderNum",
        "DifferentBenNum",
        "DifferentOrderName",
        "DifferentBenName",
        "DifferentOrderStreet",
        "DifferentBenStreet",
        "DifferentOrderZip",
        "DifferentBenZip",
    ]

    train_data_2 = train_data[cols_to_keep]
    return train_data_2

In [10]:
def create_features(df, model_dir, map_from_train_set=False):

    ## Feature Engineering

    # Hour column
    df["hour"] = df["Timestamp"].dt.hour

    # Hour frequency for each sender
    freq_by_features(
        df=df,
        level_1="Sender",
        level_2="hour",
        model_dir=model_dir,
        save_as_object=True,
        map_from_train_set=map_from_train_set,
    )
    # Hour frequency for each receiver
    freq_by_features(
        df=df,
        level_1="Receiver",
        level_2="hour",
        model_dir=model_dir,
        save_as_object=True,
        map_from_train_set=map_from_train_set,
    )
    # Sender-Currency Frequency
    freq_by_features(
        df=df,
        level_1="Sender",
        level_2="InstructedCurrency",
        model_dir=model_dir,
        save_as_object=True,
        map_from_train_set=map_from_train_set,
    )
    # Receiver-SettledCurrency Frequency
    freq_by_features(
        df=df,
        level_1="Receiver",
        level_2="SettlementCurrency",
        model_dir=model_dir,
        save_as_object=True,
        map_from_train_set=map_from_train_set,
    )
    # Sender-Receiver Frequency
    freq_by_features(
        df=df,
        level_1="Sender",
        level_2="Receiver",
        model_dir=model_dir,
        save_as_object=True,
        map_from_train_set=map_from_train_set,
    )

    # # Average Amount per Sender-Currency - not working
    # Sender_ICurrency_mean = mean_by_features(df = df, level_1="Sender" , level_2="InstructedCurrency")
    # df.loc[:,"Sender_ICurrency_mean"] = \
    #     df.loc[:,"Sender_InstructedCurrency"].map(Sender_ICurrency_mean)

    # Numbering the transactions within a account order - ben - date combination
    df = df.sort_values(
        by=[
            "SettlementDate",
            "Sender",
            "Receiver",
            "Account_order",
            "Account_ben",
            "Timestamp",
        ],
        ascending=True,
    )
    df["seq"] = (
        df.groupby(
            [
                "SettlementDate",
                "Sender",
                "Receiver",
                "Account_order",
                "Account_ben",
            ]
        ).cumcount()
        + 1
    )
    df["seq"] = df["seq"].replace(np.NAN, 1)

    # Flag columns for transactions with missing bank details
    df[["MissingBenAccount"]] = 0
    df.loc[df["Flags_ben"].isnull(), "MissingBenAccount"] = 1
    df[["MissingOrdAccount"]] = 0
    df.loc[df["Flags_order"].isnull(), "MissingOrdAccount"] = 1

    # Different sender account number from bank details
    df["DifferentOrderNum"] = np.where(
        df["Account_order"] == df["OrderingAccount"], 0, 1
    )
    # Different receiver account number from bank details
    df["DifferentBenNum"] = np.where(
        df["Account_ben"] == df["BeneficiaryAccount"], 0, 1
    )

    # Different sender account name from bank details
    df["DifferentOrderName"] = np.where(
        df["Name_order"] == df["OrderingName"], 0, 1
    )
    # Different receiver account name from bank details
    df["DifferentBenName"] = np.where(
        df["Name_ben"] == df["BeneficiaryName"], 0, 1
    )

    # Different sender account ordering street from bank details
    df["DifferentOrderStreet"] = np.where(
        df["Street_order"] == df["OrderingStreet"], 0, 1
    )
    # Different receiver account ordering street from bank details
    df["DifferentBenStreet"] = np.where(
        df["Street_ben"] == df["BeneficiaryStreet"], 0, 1
    )

    # Different sender account country code/zip from bank details
    df["DifferentOrderZip"] = np.where(
        df["CountryCityZip_order"] == df["OrderingCountryCityZip"], 0, 1
    )
    # Different receiver account country code/zip from bank details
    df["DifferentBenZip"] = np.where(
        df["CountryCityZip_ben"] == df["BeneficiaryCountryCityZip"], 0, 1
    )

    # Some missing value treatment
    df.loc[df["Flags_ben"].isna(), "Flags_ben"] = 99

    return df

# Model Building

In [11]:
# import NN layers and other componenets.
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, Dropout
from tensorflow.keras import optimizers
tf.random.set_seed(13) # to make sure the experiment is reproducible.
tf.debugging.set_log_device_placement(False)

def build_model1_two_hidden_layers(normed_train_data):
    # Keras model object created from Sequential class. This will be the container that contains all layers.
    model = Sequential()

    # The model so far is empty. It can be constructed by adding layers and compilation.
    # This Keras model with multiple hidden layers.
    
    # Input Layer with 10 Neurons
    model.add(Dense(128, input_shape = (normed_train_data.shape[1],)))    # Input layer => input_shape must be explicitly designated
    
#     model.add(Activation('relu')) # relu or sigmoid.
#     model.add(Dense(128,Activation('relu')))                         # Hidden layer 1 => only output dimension should be designated (output dimension = # of Neurons = 50)
    
    model.add(Dense(1))                          # Output layer => output dimension = 1 since it is a regression problem
    
    # Activation: sigmoid, softmax, tanh, relu, LeakyReLU. 
    #Optimizer: SGD, Adam, RMSProp, etc. # https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
    learning_rate = 0.0001
    optimizer = optimizers.Nadam(learning_rate)
#     optimizer = 'adam'
    model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy',tf.keras.metrics.Precision(),
                                                                       tf.keras.metrics.Recall()]) 
    # for regression problems, mean squared error (MSE) is often employed
    
    return model

### Split into 2 clients using Sender data

In [12]:
trainloaders=[]
valloaders=[]

In [13]:
trainset['Sender'].value_counts()

DPSUFRPP    1965697
WVOLDEMM     700002
ZOUOGB22     529859
WITAZAJJ     514627
ABVVUS6S     367029
MRQHUS44     274065
CBLHESMM      80990
FAMAPEPL      73733
HNYPITMM      49092
BRRGPTPL      36690
XAMTZWHX      33503
DECKJPJJ      30108
GDYCCHZZ      24060
KNPVECEQ       9547
MBYOTRIS       2699
UNUSAL22         24
Name: Sender, dtype: int64

Sender 'DPSUFRPP' contains almost 42% of the data. Hence they are considered as one client and the rest as another client

In [14]:
from sklearn.model_selection import train_test_split
trainloadset,valloadset=train_test_split(trainset,test_size=0.3)

In [15]:
trainloaders.append(trainloadset[trainloadset['Sender']=='DPSUFRPP'])
trainloaders.append(trainloadset[trainloadset['Sender']!='DPSUFRPP'])

In [16]:
valloaders.append(valloadset[valloadset['Sender']=='DPSUFRPP'])
valloaders.append(valloadset[valloadset['Sender']!='DPSUFRPP'])

In [17]:
testloader=testset

In [18]:
(trainloaders[0].shape)

(1376172, 31)

In [19]:
testloader.shape

(705108, 31)

### Fit / Evaluate / Predict

In [20]:
def train(model, trainloader,valloader,train_labels,valid_labels, epochs: int, verbose=False):
    """Train the network on the training set."""
    batch_size=128
    EPOCHS=2
    normed_train_data=trainloader
    normed_valid_dataset=valloader
    history = model.fit(
        normed_train_data, 
        train_labels,
        batch_size = batch_size,
        epochs=EPOCHS, 
        verbose=1,
        shuffle=True,
        steps_per_epoch = int(normed_train_data.shape[0] / batch_size) ,
        validation_data = (normed_valid_dataset, valid_labels),   
    )
    


def test(model, testloader,test_labels):
    normed_test_data=testloader
    """Evaluate the network on the entire test set."""
    print('Test Split: ')
    results =  model.evaluate(normed_test_data, test_labels, verbose=2)
    print(model.metrics_names)
    print(results)
    #print("Accuracy   : {:5.2f} ".format(accuracy))


from sklearn.metrics import (accuracy_score,classification_report,confusion_matrix)
from sklearn import metrics

def score_data(model, X_train_data, Y_train_data) -> None:

    y_pred = model.predict(X_train_data)
    ynew = np.round(y_pred).astype(int)
    print(ynew)
    print("AUPRC:", metrics.average_precision_score(y_true=Y_train_data, y_score=ynew))


### Centralized result using Keras

In [21]:
tl=data_prep1(trainloadset)
vl=data_prep1(valloadset)
testl=data_prep1(testloader)

train_labels=tl.pop('Label')
val_labels=vl.pop('Label')
test_labels=testl.pop('Label')

trainloader=norm(tl)
valloader=norm(vl)
testloader=norm(testl)

model=build_model1_two_hidden_layers(trainloader)
model.save("my_model")

INFO:tensorflow:Assets written to: my_model\assets


INFO:tensorflow:Assets written to: my_model\assets


In [22]:
train(model, trainloader,valloader,train_labels,val_labels, 1)

test(model, testloader,test_labels)

score_data(model,testloader,test_labels)

Epoch 1/2
25657/25657 [==============================] - 117s 4ms/step - loss: 0.0161 - accuracy: 0.9990 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.0164 - val_accuracy: 0.9989 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/2
25657/25657 [==============================] - 113s 4ms/step - loss: 0.0156 - accuracy: 0.9990 - precision: 1.0000 - recall: 0.0012 - val_loss: 0.0149 - val_accuracy: 0.9989 - val_precision: 1.0000 - val_recall: 0.0120
Test Split: 
22035/22035 - 55s - loss: 0.0164 - accuracy: 0.9989 - precision: 0.0000e+00 - recall: 0.0000e+00 - 55s/epoch - 2ms/step
['loss', 'accuracy', 'precision', 'recall']
[0.016425862908363342, 0.998920738697052, 0.0, 0.0]
22035/22035 [==============================] - 47s 2ms/step
[[ 0]
 [-1]
 [-2]
 ...
 [-2]
 [-2]
 [-1]]
AUPRC: 0.0013424172080825011


### For Split 1

In [23]:
tl=data_prep1(trainloaders[0])
vl=data_prep1(valloaders[0])

train_labels=tl.pop('Label')
val_labels=vl.pop('Label')

trainloader=(tl)
valloader=(vl)

train(model, trainloader,valloader,train_labels,val_labels, 1)

test(model, testloader,test_labels)

score_data(model,testloader,test_labels)

C:\Users\rahul.karuparthi\AppData\Local\Temp\ipykernel_18440\328108437.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["hour"] = df["Timestamp"].dt.hour
C:\Users\rahul.karuparthi\AppData\Local\Temp\ipykernel_18440\143421382.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[str(level_1 + "_" + level_2)] = df.loc[:, level_1] + df.loc[
C:\Users\rahul.karuparthi\AppData\Local\Temp\ipykernel_18440\143421382.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

C:\Users\rahul.karuparthi\AppData\Local\Temp\ipykernel_18440\143421382.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[str(level_1 + "_" + level_2)] = df.loc[:, level_1] + df.loc[
C:\Users\rahul.karuparthi\AppData\Local\Temp\ipykernel_18440\143421382.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, str(level_1 + "_" + level_2 + "_frequency")] = df.loc[


Epoch 1/2
10751/10751 [==============================] - 50s 4ms/step - loss: 0.0216 - accuracy: 0.9986 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.0226 - val_accuracy: 0.9985 - val_precision: 0.2000 - val_recall: 0.0012
Epoch 2/2
10751/10751 [==============================] - 46s 4ms/step - loss: 0.0216 - accuracy: 0.9986 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.0226 - val_accuracy: 0.9985 - val_precision: 0.2000 - val_recall: 0.0012
Test Split: 
22035/22035 - 53s - loss: 0.0164 - accuracy: 0.9989 - precision: 0.0000e+00 - recall: 0.0000e+00 - 53s/epoch - 2ms/step
['loss', 'accuracy', 'precision', 'recall']
[0.016449233517050743, 0.998920738697052, 0.0, 0.0]
22035/22035 [==============================] - 46s 2ms/step
[[ 0]
 [-1]
 [-2]
 ...
 [-2]
 [-2]
 [-1]]
AUPRC: 0.0013526509302489225


### For Split 2

In [24]:
tl=data_prep1(trainloaders[1])
vl=data_prep1(valloaders[1])

train_labels=tl.pop('Label')
val_labels=vl.pop('Label')

trainloader=(tl)
valloader=(vl)

train(model, trainloader,valloader,train_labels,val_labels, 1)

test(model, testloader,test_labels)

score_data(model,testloader,test_labels)

C:\Users\rahul.karuparthi\AppData\Local\Temp\ipykernel_18440\328108437.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["hour"] = df["Timestamp"].dt.hour
C:\Users\rahul.karuparthi\AppData\Local\Temp\ipykernel_18440\143421382.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[str(level_1 + "_" + level_2)] = df.loc[:, level_1] + df.loc[
C:\Users\rahul.karuparthi\AppData\Local\Temp\ipykernel_18440\143421382.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

C:\Users\rahul.karuparthi\AppData\Local\Temp\ipykernel_18440\143421382.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[str(level_1 + "_" + level_2)] = df.loc[:, level_1] + df.loc[
C:\Users\rahul.karuparthi\AppData\Local\Temp\ipykernel_18440\143421382.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, str(level_1 + "_" + level_2 + "_frequency")] = df.loc[


Epoch 1/2
14906/14906 [==============================] - 65s 4ms/step - loss: 0.0121 - accuracy: 0.9992 - precision: 0.2500 - recall: 6.6800e-04 - val_loss: 0.0120 - val_accuracy: 0.9992 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/2
14906/14906 [==============================] - 66s 4ms/step - loss: 0.0121 - accuracy: 0.9992 - precision: 0.2500 - recall: 6.6800e-04 - val_loss: 0.0120 - val_accuracy: 0.9992 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Test Split: 
22035/22035 - 51s - loss: 0.0164 - accuracy: 0.9989 - precision: 0.0000e+00 - recall: 0.0000e+00 - 51s/epoch - 2ms/step
['loss', 'accuracy', 'precision', 'recall']
[0.016449233517050743, 0.998920738697052, 0.0, 0.0]
22035/22035 [==============================] - 51s 2ms/step
[[ 0]
 [-1]
 [-2]
 ...
 [-2]
 [-2]
 [-1]]
AUPRC: 0.0013526509302489225
